In [1]:
from pathlib import Path
from time import sleep 

import numpy as np

import roboticstoolbox as rtb
import spatialmath as sm

from manipylator import VisualRobot
from manipylator import utils

# Load Robot
Below we load a URDF file describing Manny the robot manipulator. A URDF is a textual representation of a robot, defining the parameters we need so we can calculate the forward & reverse kinemtics. 

In [2]:
# path = Path('/workspace/robots/robot.urdf')
# path = Path('/workspace/robots/robot-ee.urdf')
path = Path('/workspace/robots/measured/robot.urdf')

manny = VisualRobot(path)
print(manny.model)

[I 05/23/25 23:35:11.495 748] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout
[Genesis] [23:35:17] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [23:35:17] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [23:35:17] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [23:35:17] [INFO] Running on [NVIDIA GeForce MX250] with backend gs.cuda. Device memory: 1.95 GB.
[Genesis] [23:35:17] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [23:35:18] [INFO] Scene <162fe1e> created.
[Genesis] [23:35:18] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <683f10d>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [23:35:18] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <37fe1f9>, morph: <gs.morphs.URDF(file='/workspace/robots/measured/robot.urdf')>, material: <gs.materials.Rigid>.
[Genesis] [23:35:21] [INFO] Building

ERobot: measured, 6 joints (RRRRRR), dynamics, geometry, collision
┌──────┬──────────────────┬───────┬──────────────────┬─────────────────────────────────────────────────────────────────────────────┐
│ link │       link       │ joint │      parent      │                             ETS: parent to link                             │
├──────┼──────────────────┼───────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────┤
│    0 │ base             │       │ BASE             │ SE3()                                                                       │
│    1 │ carriage_1       │     0 │ base             │ SE3(-7.47e-06, -2.937e-06, 0.037; -180°, -7.914e-41°, -6.361e-15°) ⊕ Rz(q0) │
│    2 │ shoulder_lift    │     1 │ carriage_1       │ SE3(-0.065, -2.469e-06, -0.055; 180°, 90°, 180°) ⊕ Rz(q1)                   │
│    3 │ elbow            │     2 │ shoulder_lift    │ SE3(0.355, -2.338e-06, -4.032e-06; -180°, -3.45e-15°, 3.069e-12°) ⊕ Rz(q2)  │
│ 

# Visualizing Manny
Using the `manny.visualizer.robot` object we can get the properties of it's links. For example, a given link's location in 3D space. Below we find the location of the end effector in when the all joint parameters are set to 0 degrees:

In [3]:
robot = manny.visualizer.robot
# robot

In [4]:
ee = robot.get_link('end_effector')
# ee

In [5]:
ee_initial_pos = ee.get_pos()
ee_initial_pos

tensor([-1.2851e-01,  1.5981e-06,  8.1101e-01], device='cuda:0')

In [6]:
ee_initial_quat = ee.get_quat()
ee_initial_quat

tensor([0.5000, 0.5000, 0.5000, 0.5000], device='cuda:0')

And we can verify that for the inverse is also true:

In [7]:
initial_qpos = robot.inverse_kinematics(
    link = ee,
    pos  = ee_initial_pos,
    quat = ee_initial_quat,
)
initial_qpos

[W 05/23/25 23:35:53.114 748] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


tensor([0., 0., 0., 0., 0., 0.], device='cuda:0')

I.e. that when setting all the joints to 0 degrees the end effector end up at `ee_initial_pos` and that if the end effector is at `ee_initial_pos` all the joints are at 0 degress.

Similarly, for an arbitrary (reachable) location in 3D space:

In [8]:
qpos = robot.inverse_kinematics(
    link = ee,
    pos  = [-4.8500e-02, -4.7547e-02,  4.0200e-01],
    quat = ee_initial_quat,
)
qpos

tensor([-1.3483,  0.7159,  1.8618, -1.9957, -1.3483, -3.1416], device='cuda:0')

# Tracing a Path in R3
Below we calculate a positions in R3 that correspond to a heart shaped curve:

In [86]:


scale = 0.15
x_offset = -0.0885
y_offset = 0.36
height = 0.13

# -0.0885,  0.2954,  0.2595
shape = np.array([[0, 0, 0],
                  [0.25, 0, 0],                  
                  
                  [0.5, 0, 0],                  
                  [0.75, 0, 0],                  

                  [1, 0, 0],
                  [1, 0.25, 0],

                  [1, 0.5, 0],
                  [1, 0.75, 0],

                  [1, 1, 0],
                  [0.75, 1, 0],
                  [0.5, 1, 0],

                  [0.25, 1, 0],
                  [0, 1, 0],
                  [0, 0.75, 0],
                  [0, 0.5, 0],
                  
                  [0, 0.25, 0],
                 ])
positions = scale * shape
positions[:,0] += x_offset
positions[:,1] += y_offset
positions[:,2] = height

positions[:3]

array([[-0.0885,  0.36  ,  0.13  ],
       [-0.051 ,  0.36  ,  0.13  ],
       [-0.0135,  0.36  ,  0.13  ]])

Each row is the (x, y, z) coordinates of a point on the curve. This is useful if we'd like to visualize the expected path of the end effector.

In [87]:
# for x, y, z in positions:
#     manny.visualizer.scene.draw_debug_sphere([x,y,z])
manny.visualizer.scene.clear_debug_objects()
manny.visualizer.scene.draw_debug_spheres(positions)



Similarly, we can calculate the joint parameters of each position:

In [88]:
robot.set_dofs_position([0, -0.2, 2.1, 1, 1.57, 0])

manny.visualizer.scene.step()
ee.get_pos()

tensor([-0.0885,  0.3720,  0.2415], device='cuda:0')

In [91]:
quat_pointing_down = [0, 0, 1, 0]

initial_pose = robot.inverse_kinematics(
        link = ee,
        pos  = positions[1],
        quat = quat_pointing_down,
        )

robot.set_dofs_position(initial_pose)
manny.visualizer.scene.step()

In [92]:
# # Reset to zero pose
# robot.set_dofs_position(initial_qpos)
# manny.visualizer.scene.step()

In [93]:
# camera = manny.visualizer.camera

# camera.start_recording()
# # [camera.render() for i in range(10)]

# # Reset to initial curve pose
# robot.set_dofs_position(initial_pose)
# manny.visualizer.scene.step()

# camera.render()

In [94]:

qs = [robot.inverse_kinematics(
        link = ee,
        pos  = pos,
        quat = quat_pointing_down,
        ) for pos in positions]
qs[:5]

[tensor([ 4.4321e-05, -7.0501e-01,  2.2857e+00,  1.4199e+00, -1.5708e+00,
         -4.8624e-05], device='cuda:0'),
 tensor([ 0.1052, -0.7012,  2.3126,  1.4430, -1.5708, -0.1052], device='cuda:0'),
 tensor([ 0.2108, -0.6995,  2.3252,  1.4539, -1.5708, -0.2108], device='cuda:0'),
 tensor([ 0.3144, -0.6995,  2.3232,  1.4519, -1.5708, -0.3144], device='cuda:0'),
 tensor([ 0.4136, -0.7003,  2.3067,  1.4363, -1.5708, -0.4136], device='cuda:0')]

In [95]:
robot.set_dofs_position(qs[0])
manny.visualizer.scene.step()
print(ee.get_pos())


tensor([-0.0885,  0.3600,  0.1300], device='cuda:0')


In [96]:
import pandas as pd
df = pd.DataFrame(qs, columns=['q1', 'q2', 'q3', 'q4', 'q5', 'q6']).astype(float)
df['q6'] = 0
df['q5'] = 1.57
df.head()
df.to_csv('square_qs.csv', index=False, float_format='%.3f')

And to visualize the poses Manny would take, we can animate the path:

In [97]:
for q in qs:
    robot.set_dofs_position(q)
    # print(robot.get_dofs_position())
    manny.visualizer.scene.step()
    # camera.render()
    
    sleep(0.5)

# camera.stop_recording(save_to_filename='video.mp4', fps=20)

In [19]:
# Reset to zero pose
robot.set_dofs_position(initial_qpos)
manny.visualizer.scene.step()
